# German Anselm Cluster Analysis

In [ ]:
import numpy as np
import scipy
import scipy.spatial
import sklearn
import sklearn.cluster
import pandas
import collections
import random

## Preprocessing

- Reading in the corpus data
- Splitting into historical/original and modernised portion
- Creating distance matrix from historical data

In [ ]:
INPUT = '../../histnorm/datasets/historical/german/german-anselm.dev.txt'
ENCODING = 'utf-8'
CORPUS_NAME = 'german-anselm'
FILTER = ('"', "'", '#', '.', ',', '(', ')', ';', '—', '/')

# Loading input file, which has the original and modernised token in each line separated by a \t
with open(INPUT, 'r', encoding=ENCODING) as infile:
    tokens_raw = [line.strip().split('\t') for line in infile]

# Filter out lines with control characters
# TODO: remove limit for testing
tokens = [token for token in tokens_raw[:250] if token[0] and not token[0].startswith(FILTER)]

In [ ]:
# Getting the original and modernised tokens and types
tokens_original = [token[0].lower() for token in tokens]
tokens_modernised = [token[1].lower() for token in tokens]

types_original = list(set(tokens_original))
types_modernised = list(set(tokens_modernised))

In [ ]:
# Token and hapax count
tokens_original_count = collections.Counter(tokens_original)
tokens_modernised_count = collections.Counter(tokens_modernised)

hapax_original_count = len([val for val in tokens_original_count.values() if val == 1])
hapax_modernised_count = len([val for val in tokens_modernised_count.values() if val == 1])

In [ ]:
# Levenshtein Distance
def levenshtein(string1, string2):
    if string1 == string2:
        return 0

    if not string2:
        return len(string1)
    if not string1:
        return len(string2)

    rows = len(string1) + 1
    cols = len(string2) + 1
    dist = [[0 for c in range(cols)] for r in range(rows)]

    for j in range(1, rows):
        dist[j][0] = j
    for i in range(1, cols):
        dist[0][i] = i

    for col in range(1, cols):
        for row in range(1, rows):
            cost = 1
            if string1[row - 1] == string2[col - 1]:
                cost = 0
            dist[row][col] = min(dist[row - 1][col] + 1, dist[row][col - 1] + 1, dist[row - 1][col - 1] + cost)

    # Enable for Debugging
    # print('\n'.join([''.join(['{:4}'.format(elem) for elem in row]) for row in dist]))
    return dist[row][col]

assert levenshtein('', '') == 0
assert levenshtein('foobar', 'foobar') == 0
assert levenshtein('foobar', 'foubar') == 1
assert levenshtein('foobar', 'fuubar') == 2
assert levenshtein('foobar', 'fuuar') == 3
assert levenshtein('foobar', '') == 6

In [ ]:
# Jaro Similarily
def jaro(string1, string2):

    length1 = len(string1)
    length2 = len(string2)
   
    if length1 == 0:
        return 0.0
    
    if string1 == string2:
        return 1.0   

    match_bound = max(length1, length2) // 2 - 1

    matches = 0  
    transpositions = 0

    flagged_1 = [] 
    flagged_2 = []

    for i in range(length1):
        upperbound = min(i + match_bound, length2 - 1)
        lowerbound = max(0, i - match_bound)
        for j in range(lowerbound, upperbound + 1):
            if string1[i] == string2[j] and j not in flagged_2:
                matches += 1
                flagged_1.append(i)
                flagged_2.append(j)
                break

    flagged_2.sort()

    for i, j in zip(flagged_1, flagged_2):
        if string1[i] != string2[j]:
            transpositions += 1

    if matches == 0:
        return 0.0

    return (1/3 * ( matches / length1 + matches / length2 + (matches - transpositions // 2) / matches))

assert jaro('', '') == 0.0
assert jaro('foobar', '') == 0.0
assert jaro('foobar', 'foobar') == 1.0
assert jaro('foobar', 'barfoo') == 0.4444444444444444
assert jaro('duane', 'dwayne') == 0.8222222222222222
assert jaro('hans', 'gruber') == 0.0

In [ ]:
# IBM (LCS-Levenshtein Normalized)

# Contractor, D., Faruquie, T. A., & Subramaniam, L. V. (2010, August). 
# Unsupervised cleansing of noisy text. 
# In Proceedings of the 23rd International Conference on Computational Linguistics:
# Posters (pp. 189-196). Association for Computational Linguistics.

from itertools import groupby

# Longest Common Substring
def longest_common_string(string1, string2):
    if string1 == string2:
        return len(string1)

    if not string1 or not string2:
        return 0
    
    rows = len(string1) + 1
    cols = len(string2) + 1
    table = [[0 for c in range(cols)] for r in range(rows)]

    longest = 0
    for col in range(cols):
        for row in range(rows):
            if col == 0 and row == 0:
                table[row][col] = 0
            if string1[row - 1] == string2[col - 1]:
                table[row][col] = table[row - 1][col - 1] + 1
                longest = max(longest, table[row][col])
            else:
                table[row][col] = 0
    
    return longest

assert longest_common_string('', '') == 0
assert longest_common_string('foobar', '') == 0
assert longest_common_string('foobar', 'foobar') == 6
assert longest_common_string('foobar', 'foo') == 3
assert longest_common_string('foobar', 'f') == 1


def lcs_ratio(string1, string2):
    if not string1 or not string2:
        return 0.0
    ratio = longest_common_string(string1, string2) / len(string1)
    return ratio

assert lcs_ratio('', '') == 0.0
assert lcs_ratio('foo', '') == 0.0
assert lcs_ratio('foobar', 'foobar') == 1.0
assert lcs_ratio('foo', 'bar') == 0.0
assert lcs_ratio('word', 'deoxyribonucleic') == 0.25


def consonant_skeleton(string, vowels='aeiouy'):
    without_vowels = ''.join([char for char in string if char not in vowels])     
    deduplicated_consonants = ''.join(char for char, _ in groupby(without_vowels))
    return deduplicated_consonants

assert consonant_skeleton('') == ''
assert consonant_skeleton('aeio') == ''
assert consonant_skeleton('foobar') == 'fbr'
assert consonant_skeleton('ffoobbar') == 'fbr'
assert consonant_skeleton('barfoobar') == 'brfbr'


def ibm_similarity(string1, string2, vowels='aeiouy'):
    similarity = lcs_ratio(string1, string2) / (levenshtein (consonant_skeleton(string1, vowels), consonant_skeleton(string2, vowels)) + 1)
    return similarity

assert ibm_similarity('', '') == 0.0
assert ibm_similarity('foobar', '') == 0.0
assert ibm_similarity('foobar', 'foobar') == 1.0
assert ibm_similarity('foo', 'bar') == 0.0
assert ibm_similarity('word', 'deoxyribonucleic') == 0.03125
assert ibm_similarity('foobar', 'aeiou') == 0.041666666666666664

In [ ]:
%%time

# Compute the Pairwise Distance for each Similarity Measure

ibm_vowels = 'eiauoyēüÿjūāẏīȳäöōëȯêḡîïôėǖ'

types_original_reshaped = np.array(types_original).reshape(-1,1)
types_original_pairwise_distance_levenshtein = scipy.spatial.distance.pdist(types_original_reshaped, lambda x,y: levenshtein(str(x[0]),str(y[0])))   
types_original_pairwise_distance_jaro = scipy.spatial.distance.pdist(types_original_reshaped, lambda x,y: jaro(str(x[0]),str(y[0])))   
types_original_pairwise_distance_ibm = scipy.spatial.distance.pdist(types_original_reshaped, lambda x,y: ibm_similarity(str(x[0]),str(y[0]),ibm_vowels))

In [ ]:
%%time

# Transform the Pairwise Distance for each Similarity Measure into full similarity matrix

original_distance_matrix_levenshtein = pandas.DataFrame(scipy.spatial.distance.squareform(types_original_pairwise_distance_levenshtein), index=types_original, columns=types_original)
original_distance_matrix_jaro = pandas.DataFrame(scipy.spatial.distance.squareform(types_original_pairwise_distance_jaro), index=types_original, columns=types_original)
original_distance_matrix_ibm = pandas.DataFrame(scipy.spatial.distance.squareform(types_original_pairwise_distance_ibm), index=types_original, columns=types_original)

In [ ]:
original_distance_matrix_levenshtein

In [ ]:
original_distance_matrix_jaro

In [ ]:
original_distance_matrix_ibm

## Evaluation Functions

In [ ]:
def eval_expected_n_clusters(model):
    return len(model.items())
    
def eval_expected_avg_cluster_size(model):
    """
    Compute evaluation clustering by 
    mapping all historical tokens to their modern type
    """

    evaluation_cluster = dict()

    for token in model:
        hist = token[0].lower()
        cont = token[1].lower()
        if cont in evaluation_cluster:
            evaluation_cluster[cont].append(hist)
        else:
            evaluation_cluster[cont] = [hist]

    # Calculate average cluster size
    avg_spelling_variations = sum([len(val) for val in evaluation_cluster.values()]) / len(evaluation_cluster.values())

    return avg_spelling_variations

def eval_expected_cluster_similarity_stats(model):
    """
    Calculate inter object similarity for evaluation cluster
    """
    
    avg_similarities = []

    for cluster in model.values():
        reshaped = np.array(cluster).reshape(-1,1)
        similarity = scipy.spatial.distance.pdist(reshaped, lambda x,y: jaro(str(x[0]),str(y[0])))
        avg_similarities.append(np.mean(similarity))

    return pandas.DataFrame(
        columns=['Expected'],
        index=['Mean', 'Median', 'STD', 'VAR'],
        data=[
            np.nanmean(avg_similarities),
            np.nanmedian(avg_similarities),
            np.nanstd(avg_similarities),
            np.nanvar(avg_similarities)
        ]
    )

def eval_expected_largest_clusters(model, n=10):
    clusters = list(model.values())
    clusters.sort(key=len)    
    
    c_length = [len(cl) for cl in clusters[-n:]]
    c_tokens = [cl for cl in clusters[-n:]]
    
    return pandas.DataFrame(
        data={
            'Length': c_length,
            'Tokens': c_tokens
        }
    )

In [ ]:
def eval_n_clusters(model):
    return model.n_clusters_

def eval_random_clusters(model, n=10):

    rand_clusters = []
    for cluster_id in random.choices(np.unique(model.labels_), k=10):
        cluster = types_original_reshaped[np.nonzero(model.labels_ == cluster_id)]
        rand_clusters.append([item for sublist in cluster for item in sublist])

    return pandas.Series(rand_clusters)

def eval_largest_clusters(model, n=10):

    clusters = []

    for cluster_id in np.unique(model.labels_):
        cluster = types_original_reshaped[np.nonzero(model.labels_ == cluster_id)]
        clusters.append([item for sublist in cluster for item in sublist])
    
    clusters.sort(key=len)

    c_length = [len(cl) for cl in clusters[-n:]]
    c_tokens = [cl for cl in clusters[-n:]]
    
    return pandas.DataFrame(
        data={
            'Length': c_length,
            'Tokens': c_tokens
        }
    )

def eval_cluster_similarity_stats(model):
    """
    Calculate inter object similarity
    """

    avg_similarities = []
    for cluster_id in np.unique(model.labels_):
        cluster = types_original_reshaped[np.nonzero(model.labels_ == cluster_id)]
        similarity = scipy.spatial.distance.pdist(cluster, lambda x,y: jaro(str(x[0]),str(y[0])))
        avg_similarities.append(np.mean(similarity))

    return pandas.DataFrame(
        columns=['Actual'],
        index=['Mean', 'Median', 'STD', 'VAR'],
        data=[
            np.nanmean(avg_similarities),
            np.nanmedian(avg_similarities),
            np.nanstd(avg_similarities),
            np.nanvar(avg_similarities)
        ]
    )

def eval_avg_cluster_size(model):
    print('TODO')

## Affinity Propagation Clustering

- Damping factor (between 0.5 and 1) is the extent to which the current value is maintained relative to incoming values (weighted 1 - damping). 
- This in order to avoid numerical oscillations when updating these values (messages).

In [ ]:
%%time

damping_factor = 0.5
affinity_levenshtein_euclidean = sklearn.cluster.AffinityPropagation(
    affinity='euclidean', 
    damping=damping_factor, 
    random_state=None).fit(original_distance_matrix_levenshtein)

### Evaluation

- Manual Clustering with AVG Inter-Cluster Similarity as comparison
- Number of Clusters in AHC == Type Count (modern)
- Avg Spelling variations

In [ ]:
sklearn.metrics.silhouette_score(original_distance_matrix_levenshtein, 
                                 affinity_levenshtein_euclidean.labels_, 
                                 metric='euclidean')

## Agglomerative Hierarchical Clustering

- linkage distance threshold above which, clusters will not be merged. 
- If not None, n_clusters must be None and compute_full_tree must be True.
- Metric used to compute the linkage. Can be “euclidean”, “l1”, “l2”, “manhattan”, “cosine”, or “precomputed”. 
- If linkage is “ward”, only “euclidean” is accepted. 
- If “precomputed”, a distance matrix (instead of a similarity matrix) is needed as input for the fit method.

In [ ]:
%%time

# Parameters
linkage_method = 'single'
distance_threshold = 2

# Calculation
ahc_levenshtein_single = sklearn.cluster.AgglomerativeClustering(
    n_clusters=None, 
    distance_threshold=distance_threshold, 
    affinity='precomputed', 
    linkage=linkage_method).fit(original_distance_matrix_levenshtein)

In [ ]:
eval_random_clusters(ahc_levenshtein_single)

### Evaluation

- Manual Clustering with AVG Inter-Cluster Similarity as comparison
- Number of Clusters in AHC == Type Count (modern)
- Avg Spelling variations

In [ ]:
expected_n_clusters = eval_expected_n_clusters(evaluation_cluster)
expected_n_clusters

In [ ]:
expected_avg_cluster_size = eval_expected_avg_cluster_size(tokens)
expected_avg_cluster_size

In [ ]:
expected_stats = eval_expected_cluster_similarity_stats(evaluation_cluster)
expected_stats

In [ ]:
expected_largest_clusters = eval_expected_largest_clusters(evaluation_cluster)
expected_largest_clusters

In [ ]:
actual_n_clusters = eval_n_clusters(ahc_levenshtein_single)
actual_n_clusters

In [ ]:
actual_avg_cluster_size = eval_avg_cluster_size(ahc_levenshtein_single)
actual_avg_cluster_size

In [ ]:
actual_stats = eval_cluster_similarity_stats(ahc_levenshtein_single)
actual_stats

In [ ]:
actual_largest_clusters = eval_largest_clusters(ahc_levenshtein_single)
actual_largest_clusters

In [ ]:
pandas.concat([actual_stats,expected_stats], axis=1)

## JSON Export

In [ ]:
def add_nodes(node, parent):
    """
    Recursively build tree as dict
    """

    new_node = dict(node_id=node.id, children=[], distance=node.dist)
    parent['children'].append(new_node)
    if node.left: add_nodes(node.left, new_node)
    if node.right: add_nodes(node.right, new_node)

def add_labels(node):
    """
    Recursively add labels to the tree
    """
    is_leaf = len(node['children']) == 0

    if is_leaf:
        node['name'] = labels[node['node_id']]
    else:
        list(map(add_labels, node['children']))
    del node['node_id']

if not os.path.exists(CORPUS_NAME + '-cluster-original.json'):
    # Transforming Cluster into JSON Tree,
    scipy_tree = scipy.cluster.hierarchy.to_tree(original_clustering, rd=False)
    tree = dict(name='root', children=[], distance=scipy_tree.dist)

    add_nodes(scipy_tree, tree)
    add_labels(tree['children'][0])

    with open(CORPUS_NAME + '-cluster-original.json', 'w') as clustering:
        dump(tree, clustering, indent=1)